# Chapter 40: Performance Under Production Constraints

Production environments impose constraints that laboratory benchmarks rarely replicate: network partitions occur during peak traffic, single rows become contention hot spots under viral load spikes, and thundering herds overwhelm circuit breakers during recovery. This chapter addresses the architectural patterns and defensive configurations required to maintain throughput and availability when ideal conditions deteriorate into resource contention and partial failures.

## 40.1 Hot Spots and Contention Patterns

Database contention manifests when concurrent transactions compete for the same resources—rows, pages, indexes, or sequence values. Identifying and mitigating these patterns separates architectures that scale linearly from those that collapse under load.

### 40.1.1 Row-Level Contention (The Update Hotspot)

The most common production bottleneck occurs when many transactions attempt to update the same row simultaneously—a counter table, inventory stock, or user session state.

```sql
-- Anti-pattern: Centralized counter update
UPDATE visit_counters SET count = count + 1 WHERE page_id = '/home';
-- Under 1000 concurrent requests, this serializes through a single row lock
-- Result: Lock waits cascade, connection pool exhaustion, timeout cascades
```

**Diagnostic Identification:**
```sql
-- Detect row-level contention in real-time
SELECT 
    pid,
    wait_event_type,
    wait_event,
    blocking_pid,
    blocking_lock_mode,
    blocked_lock_mode,
    query
FROM pg_stat_activity sa
LEFT JOIN pg_locks blocking ON (
    sa.pid = blocking.pid AND 
    blocking.granted = false
)
WHERE sa.wait_event_type = 'Lock'
  AND sa.wait_event = 'transactionid';
-- Multiple sessions waiting on the same transactionid indicates row contention
```

**Mitigation Strategies:**

1. **Partitioned Counters** (Eventual Consistency):
```sql
-- Create per-session counters that aggregate periodically
CREATE TABLE visit_counter_shards (
    shard_id int,
    page_id text,
    count int,
    PRIMARY KEY (shard_id, page_id)
);

-- Application determines shard via hash(client_id) % 10
INSERT INTO visit_counter_shards (shard_id, page_id, count) 
VALUES (4, '/home', 1)
ON CONFLICT (shard_id, page_id) 
DO UPDATE SET count = visit_counter_shards.count + 1;

-- Background job aggregates to master table every minute
INSERT INTO visit_counters (page_id, count)
SELECT page_id, SUM(count) FROM visit_counter_shards GROUP BY page_id
ON CONFLICT (page_id) DO UPDATE SET count = visit_counters.count + EXCLUDED.count;

-- Truncate shards after aggregation (fast, minimal logging)
TRUNCATE visit_counter_shards;
```

2. **Slotted Updates** (Deterministic Distribution):
```sql
-- Use modulo arithmetic to distribute updates across 100 slots
UPDATE inventory_slots 
SET reserved_count = reserved_count + 1 
WHERE product_id = 123 
  AND slot_id = (pg_backend_pid() % 100);

-- Sum at read time (slight computational cost, massive write scalability)
SELECT product_id, SUM(reserved_count) as total_reserved
FROM inventory_slots 
WHERE product_id = 123
GROUP BY product_id;
```

3. **Advisory Locks for Application-Level Serialization**:
When business logic requires strict serialization but you want to avoid database row locks:

```sql
-- Acquire advisory lock before update (allows queuing in application logic)
SELECT pg_try_advisory_lock(hashtext('inventory:' || product_id));

-- If acquired, proceed with update
UPDATE inventory SET stock = stock - 1 WHERE product_id = 123;

-- Release in same session or transaction end
SELECT pg_advisory_unlock(hashtext('inventory:' || product_id));
```

### 40.1.2 Index Contention (Right-Growing Indexes)

B-tree indexes on sequential values (timestamps, serial IDs) create "hotspots" at the rightmost leaf page where all inserts converge.

```sql
-- High-contention index pattern
CREATE TABLE events (
    event_id bigserial PRIMARY KEY,  -- Sequential, right-growing
    created_at timestamptz DEFAULT now(),  -- Also sequential
    payload jsonb
);
-- 10,000 inserts/sec all compete for the same leaf page latch
```

**Solutions:**

1. **Hash Sharding for PK**:
```sql
-- Use UUIDv4 or hash-distributed IDs instead of sequential
CREATE TABLE events (
    event_id uuid PRIMARY KEY DEFAULT gen_random_uuid(),
    -- or: bigint generated always as (hashtext(some_unique_data)::bigint) stored
    created_at timestamptz,
    payload jsonb
);
-- Distributes writes across entire index structure
```

2. **BRIN Indexes for Time Series**:
```sql
-- For time-series where queries are range-based on insertion order
CREATE INDEX idx_events_created_brin ON events USING BRIN (created_at) 
WITH (pages_per_range = 128);
-- Tiny index size, minimal write overhead, no right-growing hotspot
```

3. **Fillfactor Tuning for Update-Heavy Tables**:
```sql
-- Leave space for HOT updates (Heap-Only Tuple) to avoid index churn
ALTER TABLE user_sessions SET (fillfactor = 70);
-- 30% free space allows updates to stay on same page, avoiding index updates
-- for non-indexed columns
```

### 40.1.3 Sequence Contention (High-Volume ID Generation)

`SERIAL` and `IDENTITY` columns use sequences that generate next values via shared memory buffers, but cache exhaustion creates contention.

```sql
-- High-throughput sequence configuration
CREATE SEQUENCE high_volume_seq 
    CACHE 1000  -- Pre-allocate 1000 values in memory per session
    NO CYCLE;

-- Default cache is 1, causing disk writes and lock contention every value
```

**Industry Pattern: Snowflake IDs** (Application-side generation):
```python
# Distributed ID generation without database sequence
# 41-bit timestamp + 10-bit node ID + 12-bit sequence
# Application generates IDs, database inserts without sequence overhead
```

## 40.2 Batch Writes, Idempotency, and Retries

Network partitions and client timeouts create ambiguity: did the write succeed or not? Production systems must handle partial failures gracefully.

### 40.2.1 Batch Insert Strategies

**Single-Statement Atomicity**:
```sql
-- Batch insert with conflict handling
INSERT INTO events (event_id, payload, processed_at)
VALUES 
    ('evt-001', '{"data": 1}', now()),
    ('evt-002', '{"data": 2}', now()),
    ('evt-003', '{"data": 3}', now())
ON CONFLICT (event_id) DO NOTHING;
-- Atomic: all succeed or none succeed
-- Idempotent: duplicate event_ids silently ignored
```

**Chunked Batches for Large Loads**:
```sql
-- Process 1000 rows at a time to prevent long transactions
DO $$
DECLARE
    batch_size CONSTANT int := 1000;
    rows_affected int;
BEGIN
    LOOP
        INSERT INTO target_table (id, data)
        SELECT id, data 
        FROM staging_table 
        WHERE processed = false
        LIMIT batch_size
        ON CONFLICT DO NOTHING;
        
        GET DIAGNOSTICS rows_affected = ROW_COUNT;
        
        UPDATE staging_table 
        SET processed = true 
        WHERE id IN (
            SELECT id FROM staging_table 
            WHERE processed = false 
            LIMIT batch_size
        );
        
        COMMIT;  -- Frequent commits prevent lock escalation
        EXIT WHEN rows_affected = 0;
        
        PERFORM pg_sleep(0.01);  -- Micro-backoff between batches
    END LOOP;
END $$;
```

### 40.2.2 Idempotency Patterns

**Exactly-Once Semantics** require unique business keys:

```sql
-- Invoice processing with idempotency key
CREATE TABLE invoice_processing (
    idempotency_key text PRIMARY KEY,
    invoice_id bigint,
    amount_cents int,
    processed_at timestamptz DEFAULT now(),
    status processing_status DEFAULT 'pending'
);

-- Client retries with same idempotency_key
INSERT INTO invoice_processing (idempotency_key, invoice_id, amount_cents)
VALUES ('req-2024-001', 12345, 10000)
ON CONFLICT (idempotency_key) 
DO UPDATE SET 
    -- Only update if previous attempt failed
    status = CASE 
        WHEN invoice_processing.status = 'failed' THEN 'pending'::processing_status
        ELSE invoice_processing.status 
    END,
    processed_at = CASE 
        WHEN invoice_processing.status = 'failed' THEN now()
        ELSE invoice_processing.processed_at 
    END
WHERE invoice_processing.status = 'failed'
RETURNING invoice_id, status, 
    (xmax = 0) as is_new_insert;  -- True if first attempt, false if conflict/update
```

**State Machine Idempotency**:
```sql
-- Ensure transitions only happen from valid prior states
UPDATE orders 
SET status = 'shipped', shipped_at = now()
WHERE order_id = 123 
  AND status = 'paid';  -- Only ship paid orders, not cancelled ones

-- Check rows_affected: 0 means invalid transition (already shipped/cancelled)
```

### 40.2.3 Retry Strategies with Exponential Backoff

Client-side retry logic prevents thundering herds:

```python
# Industry-standard retry with jitter
import random
import time

def execute_with_retry(func, max_retries=5):
    for attempt in range(max_retries):
        try:
            return func()
        except TransientError as e:
            if attempt == max_retries - 1:
                raise
            
            # Exponential backoff: 100ms, 200ms, 400ms, 800ms, 1.6s
            base_delay = (2 ** attempt) * 0.1
            
            # Full jitter prevents synchronized retries across clients
            jitter = random.uniform(0, base_delay)
            sleep_time = base_delay + jitter
            
            time.sleep(sleep_time)
```

**Database-Side Retry Queues**:
```sql
-- Retry queue with exponential backoff
CREATE TABLE retry_queue (
    id bigserial PRIMARY KEY,
    payload jsonb,
    attempt_count int DEFAULT 0,
    next_attempt_at timestamptz DEFAULT now(),
    error_message text
);

-- Worker query (index on next_attempt_at)
SELECT * FROM retry_queue 
WHERE next_attempt_at <= now()
ORDER BY next_attempt_at
LIMIT 10
FOR UPDATE SKIP LOCKED;  -- Skip items being processed by other workers

-- On failure, increment with backoff
UPDATE retry_queue 
SET attempt_count = attempt_count + 1,
    next_attempt_at = now() + (interval '1 second' * (2 ^ attempt_count)),
    error_message = 'Error details...'
WHERE id = :id;
```

## 40.3 Timeouts, Circuit Breakers, and Backpressure

Defensive timeout hierarchies prevent cascading failures when dependencies slow down.

### 40.3.1 PostgreSQL Timeout Configuration

Layered timeouts protect resources at different granularities:

```sql
-- Connection-level (applies to all statements in session)
SET statement_timeout = '30s';        -- Maximum duration for any statement
SET lock_timeout = '5s';               -- Maximum time to wait for lock acquisition
SET idle_in_transaction_session_timeout = '60s';  -- Kill idle transactions holding locks
SET tcp_keepalives_idle = 60;        -- Detect dead connections (OS level)

-- Transaction-specific (overrides session)
BEGIN;
SET LOCAL statement_timeout = '5min';  -- Allow longer for specific batch operation
-- ... batch work ...
COMMIT;
```

**Application-Driver Timeout Alignment**:
```yaml
# Application configuration (pseudocode)
database:
  connect_timeout: 10s         # TCP connection establishment
  socket_timeout: 35s          # Must be > statement_timeout (30s) + network latency
  pool:
    connection_timeout: 5s     # Wait for connection from pool
    idle_timeout: 300s         # Return idle connections to pool
    max_lifetime: 1800s       # Force reconnect to clear any leaked state
```

### 40.3.2 Circuit Breaker Patterns

When error rates exceed thresholds, fail fast to prevent resource exhaustion:

```python
# Circuit breaker state machine
class CircuitBreaker:
    def __init__(self, failure_threshold=5, recovery_timeout=30):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.state = 'CLOSED'  # CLOSED (normal), OPEN (failing), HALF_OPEN (testing)
        self.failures = 0
        self.last_failure_time = None
    
    def call(self, func, *args):
        if self.state == 'OPEN':
            if time.time() - self.last_failure_time > self.recovery_timeout:
                self.state = 'HALF_OPEN'
            else:
                raise CircuitOpenError("Database circuit open")
        
        try:
            result = func(*args)
            self.on_success()
            return result
        except DatabaseError:
            self.on_failure()
            raise
    
    def on_failure(self):
        self.failures += 1
        self.last_failure_time = time.time()
        if self.failures >= self.failure_threshold:
            self.state = 'OPEN'
            # Alert: Stop hammering the struggling database
    
    def on_success(self):
        self.failures = 0
        self.state = 'CLOSED'
```

**Database-Side Circuit Indicators**:
```sql
-- Health check query that fails fast if database struggling
SELECT 
    CASE 
        WHEN count(*) FILTER (WHERE state = 'active') > 
             (SELECT setting::int * 0.8 FROM pg_settings WHERE name = 'max_connections')
        THEN pg_raise_error('Database overloaded')
        ELSE 'healthy'
    END as health_status
FROM pg_stat_activity;
```

### 40.3.3 Backpressure Implementation

When ingestion exceeds processing capacity, shed load gracefully rather than failing catastrophically.

**Connection Pool Backpressure**:
```sql
-- Check pool saturation before accepting work
SELECT 
    numbackends as active_connections,
    (SELECT setting::int FROM pg_settings WHERE name='max_connections') as max_conn,
    (SELECT count(*) FROM pg_stat_activity WHERE wait_event_type IS NOT NULL) as waiting,
    CASE 
        WHEN numbackends > max_conn * 0.9 THEN 'reject'
        WHEN numbackends > max_conn * 0.8 THEN 'throttle'
        ELSE 'accept'
    END as admission_status
FROM pg_stat_database 
WHERE datname = current_database();
```

**Queue-Based Load Leveling**:
```sql
-- Insert into queue with depth limit
WITH queue_depth AS (
    SELECT count(*) as depth FROM ingestion_queue WHERE status = 'pending'
)
INSERT INTO ingestion_queue (payload, status)
SELECT :payload, 'pending'
FROM queue_depth
WHERE depth < 10000;  -- Reject if queue full (backpressure)

-- Check rows_affected: 0 means queue full, return 503 Service Unavailable to client
```

## 40.4 Load Testing Methodology and Pitfalls

Validation in production-like conditions reveals constraints invisible to unit tests.

### 40.4.1 Realistic Data Generation

**Statistical Distribution Matching**:
```sql
-- Generate skewed data matching production cardinality
-- (80% of traffic hits 20% of users - Pareto distribution)
WITH RECURSIVE user_skew AS (
    SELECT 
        CASE 
            WHEN random() < 0.8 THEN (random() * 1000)::int  -- Hot users
            ELSE (random() * 50000 + 1000)::int              -- Long tail
        END as user_id,
        gen_random_uuid() as event_id
    FROM generate_series(1, 1000000)
)
INSERT INTO events (user_id, event_id, created_at)
SELECT user_id, event_id, now() - (random() * interval '30 days')
FROM user_skew;
```

**Contention Simulation**:
```bash
# pgbench custom script simulating hotspot
# hotspot.sql
\set aid random(1, 100)  -- Only 100 accounts (high contention)
BEGIN;
UPDATE pgbench_accounts SET abalance = abalance + 1 WHERE aid = :aid;
END;

# Run with high concurrency
pgbench -c 100 -j 10 -T 300 -f hotspot.sql -P 1
```

### 40.4.2 Common Load Testing Pitfalls

1. **Uniform Randomness** (The Testing Trap):
   ```sql
   -- WRONG: Uniform distribution hides lock contention
   WHERE user_id = (random() * 1000000)::int
   
   -- RIGHT: Pareto distribution reveals real hotspots
   WHERE user_id = CASE WHEN random() < 0.5 THEN 1 ELSE (random()*100000)::int END
   ```

2. **Empty Buffer Cache** (Cold Start Bias):
   - Production runs with hot cache; testing from cold shows disk-bound performance that doesn't reflect steady state
   - Warm up: Run 5 minutes of traffic before measuring

3. **Unrealistic Transaction Sizes**:
   ```sql
   -- WRONG: Single row operations only
   UPDATE accounts SET balance = balance + 1 WHERE id = 1;
   
   -- RIGHT: Mix of operations matching production ratios
   -- 80% reads, 15% single updates, 5% batch operations
   ```

4. **Ignoring Network Latency**:
   - Testing on localhost eliminates round-trip time (RTT)
   - Production RTT of 1-5ms dramatically changes transaction throughput
   - Use traffic control (`tc`) to simulate latency: `tc qdisc add dev eth0 root netem delay 5ms`

5. **Vacuum Interference**:
   - Long tests without vacuum monitoring show improving performance as bloat grows, then sudden cliffs
   - Monitor `pg_stat_progress_vacuum` during test to correlate dips with maintenance

### 40.4.3 Metrics to Capture During Load Tests

```sql
-- Snapshot script run every 10 seconds during test
SELECT 
    now() as sample_time,
    (SELECT count(*) FROM pg_stat_activity WHERE state = 'active') as active_queries,
    (SELECT count(*) FROM pg_stat_activity WHERE wait_event_type = 'Lock') as lock_waits,
    (SELECT count(*) FROM pg_stat_activity WHERE wait_event_type = 'IO') as io_waits,
    (SELECT sum(xact_commit + xact_rollback) FROM pg_stat_database) as total_xacts,
    (SELECT sum(blks_hit) FROM pg_stat_database) as cache_hits,
    (SELECT sum(blks_read) FROM pg_stat_database) as disk_reads,
    (SELECT count(*) FROM pg_stat_activity WHERE state = 'idle in transaction') as idle_in_tx;
```

**Throughput vs Latency Curves**:
Plot latency percentiles (p50, p95, p99, p999) against throughput. Healthy systems show flat latency until saturation point, then graceful degradation. Unhealthy systems show latency spikes early due to contention.

## Chapter Summary

In this chapter, you learned:

1. **Hot Spot Mitigation**: Distribute write contention using sharded counters, modulo-based row distribution, or advisory locks. Avoid sequential ID hotspots with UUIDs or hash sharding, and tune fillfactor for update-heavy tables to enable HOT updates.

2. **Batch Processing**: Use single-statement atomicity with `ON CONFLICT` for idempotent inserts. Chunk large operations to prevent long transactions and lock escalation. Implement client-side exponential backoff with jitter to prevent thundering herds during recovery.

3. **Idempotency Architecture**: Design for exactly-once semantics using database-enforced unique keys (idempotency tokens) and state machine transitions that validate prior states. Never rely on "check then act" patterns which race under concurrency.

4. **Defensive Timeouts**: Layer `statement_timeout`, `lock_timeout`, and `idle_in_transaction_session_timeout` to prevent runaway queries from consuming resources. Align application timeouts with database timeouts accounting for network latency.

5. **Circuit Breakers and Backpressure**: Implement fail-fast mechanisms when error rates exceed thresholds. Use queue depth monitoring and connection pool saturation metrics to shed load gracefully (HTTP 503) rather than accepting work that will timeout, wasting resources.

6. **Load Testing Realism**: Generate skewed data distributions (Pareto, not uniform) to expose lock contention. Warm caches before measuring, simulate network latency, and monitor vacuum interference. Measure latency percentiles, not just throughput, and watch for early latency spikes indicating architectural bottlenecks.

---

**Next:** In Chapter 41, we will explore JSONB in Production—covering modeling tradeoffs between JSONB and normalized tables, effective indexing strategies (GIN, expression indexes), query patterns that leverage PostgreSQL's JSON operators, and schema evolution strategies for document-oriented data within a relational system.